## Extract CERFA variables from Openfisca-France repo (and apply some manual tweaks)

In [ ]:
import pkgutil
import json

from openfisca_france import model
from openfisca_france.model.base import Variable

from deuxpots import CERFA_VARIABLES_PATH

Import all modules from OpenFisca-France model (so that __subclass__ finds them).

In [ ]:
__all__ = []
for loader, module_name, is_pkg in pkgutil.walk_packages(model.__path__):
    __all__.append(module_name)
    _module = loader.find_module(module_name).load_module(module_name)
    globals()[module_name] = _module

In [ ]:
variables = []
for cls in Variable.__subclasses__():
    try:
        cerfa = cls.cerfa_field
    except AttributeError:
        continue
    if isinstance(cerfa, dict):
        boxes = [cerfa[k] for k in sorted(cerfa)]
    elif isinstance(cerfa, str):
        boxes = [cerfa]
    else:
        raise ValueError("cerfa_field is neither a dict nor a string.")
    if len(boxes) == 1 and len(boxes[0]) == 1:  # Exclude single-letter "family" boxes (to be added manually later on)
        continue
    var_type = cls.value_type.__name__
    variable = {
        'boxes': boxes,
        'type': 'int' if var_type == 'float' else var_type,
        'description': getattr(cls, 'label', None),
    }
    variables.append(variable)

Add some new boxes from 2023, that are not present in Openfisca yet.

In [ ]:
variables += [
    {
        'boxes': ["8HV", "8IV", "8JV", "8KV"],
        'type': 'int',
        'description': "Prélèvement à la source déjà payé - retenue à la source sur les salaires et pensions",
    },
    {
        'boxes': ["8HW", "8IW", "8JW", "8KW"],
        'type': 'int',
        'description': "Prélèvement à la source déjà payé - acomptes d'impôt sur le revenu",
    },
    {
        'boxes': ["8HX", "8IX", "8JX", "8KX"],
        'type': 'int',
        'description': "Prélèvement à la source déjà payé - acomptes de prélèvements sociaux",
    },
    {
        'boxes': ["8HY", "8IY", "8JY", "8KY"],
        'type': 'int',
        'description': "Prélèvement à la source - Remboursement de trop-prélevé déjà obtenu - impôt sur le revenu",
    },
    {
        'boxes': ["8HZ", "8IZ", "8JZ", "8KZ"],
        'type': 'int',
        'description': "Prélèvement à la source déjà payé - Remboursement de trop-prélevé déjà obtenu - prélèvements sociaux",
    },
]

The family boxes (booleans or small integer) are added manually, because:
- Their codes are ambigouous (e.g. in openfisca, there are two "F" boxes).
- The codes used in the online simulator are prefixed by the section letter: A, B, C or D (e.g. "AF" instead of "F").
- Somme boxes are not grouped together whereas they should (e.g. "AP" and "AF" are the same, but for the two partners).

In [ ]:
variables += [
    # {'boxes': ['AG'], 'type': 'bool', 'description': "Titulaire d'une pension de veuve de guerre"},
    # {'boxes': ['AN'], 'type': 'bool', 'description': "Vous ne viviez pas seul au 1er janvier de l'année de perception des revenus"},
    # {'boxes': ['AL'], 'type': 'bool', 'description': "Situation pouvant donner droit à une demi-part supplémentaire: vous vivez seul au 1er janvier de l'année de perception des revenus et vous avez élevé un enfant pendant au moins 5 ans durant la période où vous viviez seul (définition depuis 2009) - Un au moins de vos enfants à charge ou rattaché est issu du mariage avec votre conjoint décédé (définition avant 2008)"},
    {'boxes': ['0AP', '0AF'], 'type': 'bool', 'description': "Titulaire d'une pension pour une invalidité d'au moins 40 % ou d'une carte d'invalidité d'au moins 80%"},
    {'boxes': ['0AW', '0AS'], 'type': 'bool', 'description': "Vous êtes âgé de plus de 74 ans, vous êtes titulaire de la carte du combattant ou d'une pension militaire d'invalidité ou de victime de guerre"},
    {'boxes': ['0CF'], 'type': 'int', 'description': "Nombre d'enfants à charge non mariés, qui ne sont pas en résidence alternée, de moins de 18 ans au 1er janvier de l'année de perception des revenus, ou nés durant la même année ou handicapés quel que soit leur âge"},
    {'boxes': ['0CG'], 'type': 'int', 'description': "Nombre d'enfants qui ne sont pas en résidence alternée à charge titulaires de la carte d'invalidité."},
    {'boxes': ['0CH'], 'type': 'int', 'description': "Nombre d'enfants à charge en résidence alternée, non mariés de moins de 18 ans au 1er janvier de l'année de perception des revenus, ou nés durant la même année ou handicapés quel que soit leur âge"},
    {'boxes': ['0CI'], 'type': 'int', 'description': "Nombre d'enfants à charge en résidence alternée titulaires de la carte d'invalidité"},
    {'boxes': ['0CR'], 'type': 'int', 'description': "Nombre de titulaires (autres que les enfants) de la carte invalidité d'au moins 80 %"},
    {'boxes': ['0DJ'], 'type': 'int', 'description': "Nombre d'enfants majeurs célibataires sans enfant"},
    {'boxes': ['0DN'], 'type': 'int', 'description': "Nombre d'enfants mariés/pacsés et d'enfants non mariés chargés de famille"},
]

The household status boxes are a particular case, because they are not prefixed in the simulator.

In [ ]:
variables += [
    {'boxes': ['M'], 'type': 'bool', 'description': "Marié·e"},
    {'boxes': ['O'], 'type': 'bool', 'description': "Pacsé·e"},
    {'boxes': ['D'], 'type': 'bool', 'description': "Divorcé·e/séparé·e"},
    {'boxes': ['C'], 'type': 'bool', 'description': "Célibataire"},
    {'boxes': ['V'], 'type': 'bool', 'description': "Veuf·ve"},
]

In [ ]:
with open(CERFA_VARIABLES_PATH, 'w+') as f:
    json.dump(variables, f, indent=2)

## Prepare for image annotation
Extract page 2 of a tax sheet as an image. This image can then be used in Labelme to annotate the coordinates of each family box to extract.

In [ ]:
import fitz
from fitz import Rect

PDF_PATH = "declaration.pdf"

doc = fitz.open(PDF_PATH)
family_page = doc[1]
family_page.get_pixmap(clip=family_page.rect).save("family_page.png")